In [1]:
import os
import numpy as np
import pandas as pd
import math
import torch
import torch.optim as optim
from torch.utils.data import random_split
from src.Datasets import CIFAR100Dataset
from utils_DGMMC import DGMMClassifier, train_from_features_PCA, test_from_features_PCA, get_means_bandwidth_from_features, CrossEntropy
from utils import get_trained_PCA

# Hyper Paremeters

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('Code running on :', device)

P = 80 # percentage of cumulative variance ration to keep
G = 1 # Number of Gaussian component in each GMMs
embedding = 'IMAGEBIND'

classes = 100
batch_size = 64
nb_epochs = 30

Code running on : cuda:0


# Paths

In [3]:
EXPERIMENT_PATH = os.path.join('experiments_tutorial_on_CIFAR100')
if os.path.isdir(EXPERIMENT_PATH) is False:
        os.mkdir(EXPERIMENT_PATH)
        
FEATURES_ABOSLUTE_PATH = os.path.join('/home/jeremy/Documents/Datasets/CIFAR100', 'Features') # Path to directory containing all the features provided by CLIP or ImageBind

embeding_folder = os.path.join(EXPERIMENT_PATH, 'IMAGEBIND')
if os.path.isdir(embeding_folder) is False:
    os.mkdir(embeding_folder)

SDGM_folder_path = os.path.join(embeding_folder, 'DGMMC')
if os.path.isdir(SDGM_folder_path) is False:
    os.mkdir(SDGM_folder_path)

results_path = os.path.join(SDGM_folder_path, 'results')
if os.path.isdir(results_path) is False:
    os.mkdir(results_path)

models_path = os.path.join(SDGM_folder_path, 'models')
if os.path.isdir(models_path) is False:
    os.mkdir(models_path)

# Datasets and Dataloaders

Custom datasets have been created for each datasets in order to load directly the features provided by the pretrained CNN (CLIP or ImageBind).

In [4]:
trainset = CIFAR100Dataset(os.path.join(FEATURES_ABOSLUTE_PATH, embedding, 'train'), train=True)
train_ds, val_ds = random_split(trainset, [math.floor(0.90*len(trainset)), len(trainset) - math.floor(0.90*len(trainset))])

trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True,num_workers = 8, pin_memory = False)
valloader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False,num_workers = 8, pin_memory = False)

testset = CIFAR100Dataset(os.path.join(FEATURES_ABOSLUTE_PATH, embedding, 'test'), train=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False,num_workers = 8, pin_memory = False)

Files already downloaded and verified
Files already downloaded and verified


# PCA decomposition

Here we use the function "get_trained_PCA" to train the PCA using the training data o the dataset. Then, we compute the cumulative variate ratio and only keep the $d$ first eigenvectors that contains $P$ of the variance.

In [5]:

pca = get_trained_PCA(trainloader, 1024)

cumsum = np.cumsum(pca.explained_variance_ratio_)

d = np.argmax(cumsum >= P/100) + 1

print(d)

100%|██████████| 704/704 [00:01<00:00, 591.91it/s]


164


# Initialisation of the model
Using our classifier DGMMC, we can initilize the means of each Gaussian around the means o each class computed using the training data of the dataset. This allows to converge much quicker and reduce training time.

In [6]:
init_means, init_stds = get_means_bandwidth_from_features(classes, trainloader, pca, d)

model = DGMMClassifier(d,classes, G, init_means)
model.to(device)

criterion = CrossEntropy()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=nb_epochs, eta_min=1e-4)
best_loss = math.inf

model_path = os.path.join(models_path, 'model.pt')

Processing : 100%|██████████| 704/704 [00:03<00:00, 180.13batchs/s]


# Training procedure

In [7]:
tr = []
val = []
for epoch in range(nb_epochs):

    model, train_loss, train_acc = train_from_features_PCA(classes, device, model, trainloader, criterion, optimizer, pca, d)
    tr.append(np.hstack((train_loss, train_acc)))

    val_loss, val_acc = test_from_features_PCA(classes, device, model, valloader, criterion, pca, d)
    val.append(np.hstack((val_loss, val_acc)))

    print("[Epoch {}/{}] tr_loss: {:.4f} -- tr_acc: {:.3f} -- val_loss: {:.4f} -- val_acc: {:.3f}".format(epoch, nb_epochs, train_loss, train_acc, val_loss, val_acc))

    if val_loss < best_loss:
        torch.save(model, model_path)
        best_loss = val_loss

    scheduler.step()

best_model = torch.load(model_path)
best_model.eval()
best_model.to(device)

test_loss, test_acc = test_from_features_PCA(classes, device, best_model, testloader, criterion, pca, d)
print("Test: test_loss: {:.5f} -- test_acc: {:.3f}".format(test_loss, test_acc))


100%|██████████| 79/79 [00:00<00:00, 138.20it/s]


[Epoch 0/30] tr_loss: 18.9126 -- tr_acc: 0.858 -- val_loss: 15.5153 -- val_acc: 0.878


100%|██████████| 79/79 [00:00<00:00, 149.38it/s]


[Epoch 1/30] tr_loss: 16.3650 -- tr_acc: 0.888 -- val_loss: 15.0756 -- val_acc: 0.883


100%|██████████| 79/79 [00:00<00:00, 127.25it/s]


...

[Epoch 28/30] tr_loss: 2.3098 -- tr_acc: 0.936 -- val_loss: 2.6204 -- val_acc: 0.888


100%|██████████| 79/79 [00:00<00:00, 140.31it/s]


[Epoch 29/30] tr_loss: 2.2462 -- tr_acc: 0.937 -- val_loss: 2.5808 -- val_acc: 0.890


100%|██████████| 157/157 [00:01<00:00, 106.63it/s]

Test: test_loss: 2.74961 -- test_acc: 0.887


# Savind the results and the training details

In [8]:
tr = np.stack(tr, axis=0)
df_tr = pd.DataFrame(tr, columns=['loss', 'acc'])
fpath = os.path.join(results_path, 'train.csv')

val = np.stack(val, axis=0)
df_val = pd.DataFrame(val, columns=['loss', 'acc'])
fpath = os.path.join(results_path, 'val.csv')
df_val.to_csv(fpath, sep=';')

te = np.vstack((test_loss, test_acc)).transpose()
df_test = pd.DataFrame(te, columns=['loss', 'acc'])
fpath = os.path.join(results_path, 'test.csv')
df_test.to_csv(fpath, sep=';')

feat_infos = np.vstack((P/100, d)).transpose()
df_feat_info = pd.DataFrame(feat_infos, columns=['P', 'Features_kept'])
fpath = os.path.join(results_path, 'Features.csv')
df_feat_info.to_csv(fpath, sep=';')